In [1]:
import platform,os
os_name = platform.system()
hdfs_home = "/user/" + os.getenv("HOME").split("/")[2]

In [2]:
print(hdfs_home)
print(os_name)

/user/kranthidr
Linux


In [3]:
path = hdfs_home+""

In [4]:
spark_home = "/usr/hdp/current/spark2-client"

In [5]:
import findspark
findspark.init(spark_home)
#findspark.init()
findspark.find()

'/usr/hdp/current/spark2-client'

In [6]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("yarn").appName("sg15-runSparkInCluster").getOrCreate()
#spark = SparkSession.builder.master("local[*]").appName("sg13-rddAdvanced").getOrCreate()

In [7]:
sc = spark.sparkContext

In [8]:
for x in sc._conf.getAll():
    if '/proxy/' in x[1]:
        print(x[1])

http://rm01.itversity.com:19288/proxy/application_1538287994192_0195


In [9]:
df1 = spark.range(2, 10000000, 2)

In [10]:
df2 = spark.range(2, 10000000, 4)

In [11]:
step1 = df1.repartition(5)

In [12]:
step12 = df2.repartition(6)

In [13]:
step2 = step1.selectExpr("id * 5 as id")

In [14]:
step3 = step2.join(step12, ["id"])

In [15]:
step4 = step3.selectExpr("sum(id)")

In [16]:
step4.collect() # 2500000000000

[Row(sum(id)=2500000000000)]

In [17]:
step4.explain()

== Physical Plan ==
*(7) HashAggregate(keys=[], functions=[sum(id#6L)])
+- Exchange SinglePartition
   +- *(6) HashAggregate(keys=[], functions=[partial_sum(id#6L)])
      +- *(6) Project [id#6L]
         +- *(6) SortMergeJoin [id#6L], [id#2L], Inner
            :- *(3) Sort [id#6L ASC NULLS FIRST], false, 0
            :  +- Exchange hashpartitioning(id#6L, 200)
            :     +- *(2) Project [(id#0L * 5) AS id#6L]
            :        +- Exchange RoundRobinPartitioning(5)
            :           +- *(1) Range (2, 10000000, step=2, splits=2)
            +- *(5) Sort [id#2L ASC NULLS FIRST], false, 0
               +- Exchange hashpartitioning(id#2L, 200)
                  +- Exchange RoundRobinPartitioning(6)
                     +- *(4) Range (2, 10000000, step=4, splits=2)
